In [1]:
import pandas as pd
import os
import shutil 
import numpy as np
import torch
import torch.nn as nn
import torchvision.transforms.functional as F
import torch.optim as optim
import torchvision
from torch.utils.data.sampler import SubsetRandomSampler
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt # for plotting
import random

In [2]:
#path
from google.colab import drive
drive.mount('/content/drive')
raw_data_path = "/content/drive/My Drive/MIE424/Exploring-Optimization-in-Deep-Learning-with-Various-Algorithms/Data/OrganizedDataSet/"
tensor_data_path = "/content/drive/My Drive/MIE424/Exploring-Optimization-in-Deep-Learning-with-Various-Algorithms/Data/Tensor/"

Mounted at /content/drive


In [3]:
def random_transform(img_tensor, pre_transform):
  #randomly perform one of the augmentation technique (thats diff from the pre one)
  augmentation_indecator = random.randint(0, 3)
  while augmentation_indecator == pre_transform:
      augmentation_indecator = random.randint(0, 3)

  #brightness
  if augmentation_indecator == 0:
    #adjuest brightness between [0.5, 1.5], not including 1 (original brightness)
    brightness_adj_degree = random.uniform(0.5, 1.5)
    while brightness_adj_degree == 1.0:
      brightness_adj_degree = random.uniform(0.5, 1.5)
    image_new = F.adjust_brightness(img_tensor, brightness_adj_degree)
  
  #rotate
  elif augmentation_indecator == 1:
    #rotaion between [0, 15] degree, not including 0 (original)
    rotation_degree = random.uniform(0, 15)
    while rotation_degree == 0.0:
      rotation_degree = random.uniform(0, 15)
    image_new = F.rotate(img_tensor, rotation_degree)

  #verti flip
  elif augmentation_indecator == 2:
    image_new = F.vflip(img_tensor)
  
  #hori flip
  else:
    image_new = F.hflip(img_tensor)

  return image_new, augmentation_indecator



In [4]:
def transform_data_and_save(image_datapath, tensor_datapath):
    raw_transform = transforms.Compose([
            #resize to 180 * 180 
            transforms.Resize([180,180]), 
            #transform to tensor
            transforms.ToTensor()
        ])
    dataset = ImageFolder(root = image_datapath, transform = raw_transform)
    print("finished transforming data, size of dataset: ", len(dataset))
    print("starting to save data")
    #data[0] --> tensor; data[1] --> label
    for i, data in enumerate(dataset):
      if data[1] == 0:
        #random transform 1:
        img_transform_1, transform_1 = random_transform(data[0], -1)
        #random transform 2:
        img_transform_2, transform_2 = random_transform(data[0], transform_1)

        #name for three images
        raw_img_name = tensor_data_path + "Covid Negative/raw_" + str(i) + '.pt'
        transform1_img_name = tensor_data_path + "Covid Negative/transform_1_" + str(i) + '.pt'
        transform2_img_name = tensor_data_path + "Covid Negative/transform_2_" + str(i) + '.pt'
        # print(img_transform_1, transform_1)
        # print(img_transform_2, transform_2)
        # print(data[0])
        # print(len(img_transform_2[0]))
        # break

        #store three tensors
        torch.save(data[0], raw_img_name)
        torch.save(img_transform_1, transform1_img_name)
        torch.save(img_transform_2, transform2_img_name)
      else:
        #random transform 1:
        img_transform_1, transform_1 = random_transform(data[0], -1)
        #random transform 2:
        img_transform_2, transform_2 = random_transform(data[0], transform_1)

        #name for three images
        raw_img_name = tensor_data_path + "Covid Positive/raw_" + str(i) + '.pt'
        transform1_img_name = tensor_data_path + "Covid Positive/transform_1_" + str(i) + '.pt'
        transform2_img_name = tensor_data_path + "Covid Positive/transform_2_" + str(i) + '.pt'
        
        #store three tensor
        torch.save(data[0], raw_img_name)
        torch.save(img_transform_1, transform1_img_name)
        torch.save(img_transform_2, transform2_img_name)
      if i % 100 == 0:
        print("At: ", i)
    # indices = list(range(len(dataset)))
    # np.random.seed(1000) # Fixed numpy random seed for reproducible shuffling
    # np.random.shuffle(indices)
    # test_set = test_size
    # validation_split = int(trian_val_split * (len(indices) - test_size))
    # print(test_set,validation_split, (len(indices) - test_size) )
    # train_indices , valid_indices, test_indeices = indices[:validation_split], indices[validation_split:(len(indices) - test_size)], indices[(len(indices) - test_size):]
    print("finished saving data")
    return dataset

In [ ]:
dataset = transform_data_and_save(raw_data_path, tensor_data_path)

finished transforming data, size of dataset:  692
starting to save data
At:  0
At:  100
At:  200
At:  300
At:  400
At:  500
At:  600
finished saving data


In [5]:
#test loading 
testdataset = torchvision.datasets.DatasetFolder(tensor_data_path, loader=torch.load, extensions=('.pt'))
len(testdataset)

2076

In [ ]:
testdataset.class_to_idx

{'Covid Negative': 0, 'Covid Positive': 1}

In [ ]:
testdataset

Dataset DatasetFolder
    Number of datapoints: 2076
    Root location: /content/drive/My Drive/MIE424/Data/Tensor/

In [ ]:
testdataset[0]

(tensor([[[0.2549, 0.2667, 0.2667,  ..., 0.0039, 0.0039, 0.0039],
          [0.2902, 0.3098, 0.3373,  ..., 0.0039, 0.0039, 0.0039],
          [0.2235, 0.2745, 0.5333,  ..., 0.0039, 0.0039, 0.0039],
          ...,
          [0.0039, 0.0039, 0.0039,  ..., 0.0039, 0.0000, 0.0000],
          [0.0039, 0.0039, 0.0039,  ..., 0.0039, 0.0000, 0.0039],
          [0.0039, 0.0039, 0.0039,  ..., 0.0039, 0.0039, 0.0039]],
 
         [[0.2549, 0.2667, 0.2667,  ..., 0.0039, 0.0039, 0.0039],
          [0.2902, 0.3098, 0.3373,  ..., 0.0039, 0.0039, 0.0039],
          [0.2235, 0.2745, 0.5333,  ..., 0.0039, 0.0039, 0.0039],
          ...,
          [0.0039, 0.0039, 0.0039,  ..., 0.0039, 0.0000, 0.0000],
          [0.0039, 0.0039, 0.0039,  ..., 0.0039, 0.0000, 0.0039],
          [0.0039, 0.0039, 0.0039,  ..., 0.0039, 0.0039, 0.0039]],
 
         [[0.2549, 0.2667, 0.2667,  ..., 0.0039, 0.0039, 0.0039],
          [0.2902, 0.3098, 0.3373,  ..., 0.0039, 0.0039, 0.0039],
          [0.2235, 0.2745, 0.5333,  ...,